# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/02_metriques_evaluation/02_demo_metriques_regression.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '02_demo_metriques_regression.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 02 - Métriques de Régression

**Objectifs :**
- Comprendre et calculer les métriques de régression (MSE, RMSE, MAE, R², MAPE)
- Visualiser les erreurs de prédiction
- Comparer différents modèles avec les bonnes métriques
- Identifier l'impact des outliers sur les métriques
- Savoir choisir la bonne métrique selon le contexte

**Dataset :** Diabetes Dataset (régression continue)

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)

# Configuration des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

# Seed pour reproductibilité
np.random.seed(42)

## 1. Chargement et Exploration des Données

In [ ]:
# Chargement du dataset Diabetes
data = load_diabetes()
X = data.data
y = data.target  # Mesure quantitative de progression du diabète après 1 an

print("Dataset Diabetes")
print(f"Nombre d'exemples : {X.shape[0]}")
print(f"Nombre de features : {X.shape[1]}")
print(f"\nFeatures : {data.feature_names}")
print(f"\nCible (y) : Progression du diabète")
print(f"  - Min : {y.min():.2f}")
print(f"  - Max : {y.max():.2f}")
print(f"  - Moyenne : {y.mean():.2f}")
print(f"  - Écart-type : {y.std():.2f}")

In [ ]:
# Visualisation de la distribution de la cible
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme
axes[0].hist(y, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(y.mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne = {y.mean():.2f}')
axes[0].set_xlabel('Progression du diabète', fontsize=12)
axes[0].set_ylabel('Fréquence', fontsize=12)
axes[0].set_title('Distribution de la Variable Cible', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Boxplot
axes[1].boxplot(y, vert=True, patch_artist=True, 
                boxprops=dict(facecolor='lightgreen', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Progression du diabète', fontsize=12)
axes[1].set_title('Boxplot de la Variable Cible', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Observation : La distribution est relativement normale, sans outliers extrêmes.")

## 2. Entraînement de Modèles de Régression

In [ ]:
# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"Train set : {X_train.shape[0]} exemples")
print(f"Test set : {X_test.shape[0]} exemples")

In [ ]:
# Entraînement de plusieurs modèles
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (alpha=1)': Ridge(alpha=1.0),
    'Lasso (alpha=1)': Lasso(alpha=1.0)
}

# Dictionnaire pour stocker les prédictions
predictions = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    print(f"✅ {name} entraîné")

print("\nTous les modèles sont entraînés !")

## 3. Calcul Manuel des Métriques

Calculons les métriques à la main pour bien comprendre les formules.

In [ ]:
# Utilisons les prédictions du modèle Linear Regression
y_pred = predictions['Linear Regression']

# Calcul manuel des métriques
n = len(y_test)

# Erreurs
errors = y_test - y_pred

# MSE : Mean Squared Error
mse_manual = np.sum(errors ** 2) / n

# RMSE : Root Mean Squared Error
rmse_manual = np.sqrt(mse_manual)

# MAE : Mean Absolute Error
mae_manual = np.sum(np.abs(errors)) / n

# R² : Coefficient de Détermination
ss_res = np.sum(errors ** 2)  # Somme des carrés des résidus
ss_tot = np.sum((y_test - y_test.mean()) ** 2)  # Variance totale
r2_manual = 1 - (ss_res / ss_tot)

# MAPE : Mean Absolute Percentage Error (en %)
mape_manual = np.mean(np.abs(errors / y_test)) * 100

print("=" * 70)
print("MÉTRIQUES CALCULÉES MANUELLEMENT (Linear Regression)")
print("=" * 70)
print(f"\nMSE  = (1/n) * Σ(y - ŷ)²")
print(f"     = (1/{n}) * {ss_res:.2f}")
print(f"     = {mse_manual:.4f}")
print(f"\nRMSE = √(MSE)")
print(f"     = √({mse_manual:.4f})")
print(f"     = {rmse_manual:.4f}")
print(f"\nMAE  = (1/n) * Σ|y - ŷ|")
print(f"     = {mae_manual:.4f}")
print(f"\nR²   = 1 - (SS_res / SS_tot)")
print(f"     = 1 - ({ss_res:.2f} / {ss_tot:.2f})")
print(f"     = {r2_manual:.4f}")
print(f"\nMAPE = (100/n) * Σ|y - ŷ| / y")
print(f"     = {mape_manual:.4f} %")
print("=" * 70)

In [ ]:
# Vérification avec scikit-learn
print("\n" + "=" * 70)
print("VÉRIFICATION AVEC SCIKIT-LEARN")
print("=" * 70)
print(f"MSE  (sklearn) = {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE (sklearn) = {mean_squared_error(y_test, y_pred, squared=False):.4f}")
print(f"MAE  (sklearn) = {mean_absolute_error(y_test, y_pred):.4f}")
print(f"R²   (sklearn) = {r2_score(y_test, y_pred):.4f}")
print(f"MAPE (sklearn) = {mean_absolute_percentage_error(y_test, y_pred) * 100:.4f} %")
print("=" * 70)
print("\n✅ Les calculs manuels correspondent parfaitement à scikit-learn !")

## 4. Interprétation des Métriques

In [ ]:
print("\n" + "=" * 70)
print("INTERPRÉTATION DES MÉTRIQUES")
print("=" * 70)

print(f"\n📊 MSE = {mse_manual:.2f}")
print("   → Unité : (progression)² (difficile à interpréter)")
print("   → Pénalise fortement les grandes erreurs (à cause du carré)")
print("   → Sensible aux outliers")

print(f"\n📊 RMSE = {rmse_manual:.2f}")
print("   → Unité : progression (même unité que y)")
print(f"   → Interprétation : En moyenne, les prédictions s'écartent de {rmse_manual:.2f} unités")
print("   → Plus facile à interpréter que MSE")

print(f"\n📊 MAE = {mae_manual:.2f}")
print("   → Unité : progression (même unité que y)")
print(f"   → Interprétation : Erreur absolue moyenne de {mae_manual:.2f} unités")
print("   → Moins sensible aux outliers que RMSE")
print(f"   → Écart RMSE - MAE = {rmse_manual - mae_manual:.2f} (faible = peu d'outliers)")

print(f"\n📊 R² = {r2_manual:.4f}")
print(f"   → Interprétation : Le modèle explique {r2_manual * 100:.2f}% de la variance")
print(f"   → Les {(1 - r2_manual) * 100:.2f}% restants sont dus au bruit ou variables non capturées")
print("   → R² = 0 : modèle équivalent à prédire la moyenne")
print("   → R² = 1 : modèle parfait (rare en pratique)")

print(f"\n📊 MAPE = {mape_manual:.2f}%")
print(f"   → Interprétation : Erreur moyenne de {mape_manual:.2f}% par rapport aux vraies valeurs")
print("   → Indépendant de l'échelle (utile pour comparer différents datasets)")
print("   → Attention : non défini si y contient des 0")

## 5. Comparaison des Modèles

In [ ]:
# Calcul des métriques pour tous les modèles
results = []

for name, y_pred in predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    results.append({
        'Modèle': name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE (%)': mape
    })

df_results = pd.DataFrame(results)

print("\n" + "=" * 90)
print("COMPARAISON DES MODÈLES")
print("=" * 90)
print(df_results.to_string(index=False))
print("\n💡 Plus petit = meilleur pour MSE, RMSE, MAE, MAPE")
print("💡 Plus grand = meilleur pour R² (max = 1)")

In [ ]:
# Visualisation comparative des métriques
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
metrics_to_plot = ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE (%)']
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

for i, (metric, color) in enumerate(zip(metrics_to_plot, colors)):
    ax = axes[i // 3, i % 3]
    
    bars = ax.bar(df_results['Modèle'], df_results[metric], color=color, alpha=0.7, edgecolor='black')
    
    # Mettre en évidence le meilleur modèle
    if metric == 'R²':
        best_idx = df_results[metric].idxmax()
    else:
        best_idx = df_results[metric].idxmin()
    
    bars[best_idx].set_color('gold')
    bars[best_idx].set_edgecolor('red')
    bars[best_idx].set_linewidth(3)
    
    ax.set_ylabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'Comparaison - {metric}', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=15)
    ax.grid(alpha=0.3, axis='y')
    
    # Ajouter les valeurs sur les barres
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontsize=9)

# Supprimer le subplot vide
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

print("\n⭐ Modèle en OR = meilleur pour cette métrique")

## 6. Visualisation des Prédictions vs Vraies Valeurs

In [ ]:
# Scatter plot : prédictions vs vraies valeurs
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, y_pred) in enumerate(predictions.items()):
    ax = axes[i]
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    
    # Ligne de référence (prédiction parfaite : y = ŷ)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Prédiction parfaite')
    
    # Métriques
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    ax.set_xlabel('Vraies Valeurs', fontsize=12)
    ax.set_ylabel('Prédictions', fontsize=12)
    ax.set_title(f'{name}\nR² = {r2:.3f}, RMSE = {rmse:.2f}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterprétation :")
print("  - Points proches de la ligne rouge : bonnes prédictions")
print("  - Points au-dessus de la ligne : surestimation")
print("  - Points en-dessous de la ligne : sous-estimation")

## 7. Distribution des Erreurs (Residuals)

In [ ]:
# Analyse des résidus (erreurs) pour chaque modèle
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, (name, y_pred) in enumerate(predictions.items()):
    residuals = y_test - y_pred
    
    # Histogramme des résidus
    ax1 = axes[0, i]
    ax1.hist(residuals, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Erreur nulle')
    ax1.axvline(residuals.mean(), color='green', linestyle='--', linewidth=2, 
                label=f'Moyenne = {residuals.mean():.2f}')
    ax1.set_xlabel('Résidus (y - ŷ)', fontsize=11)
    ax1.set_ylabel('Fréquence', fontsize=11)
    ax1.set_title(f'{name} - Distribution des Résidus', fontsize=12, fontweight='bold')
    ax1.legend(fontsize=9)
    ax1.grid(alpha=0.3)
    
    # Résidus vs Prédictions (détection de patterns)
    ax2 = axes[1, i]
    ax2.scatter(y_pred, residuals, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    ax2.axhline(0, color='red', linestyle='--', linewidth=2)
    ax2.set_xlabel('Prédictions', fontsize=11)
    ax2.set_ylabel('Résidus', fontsize=11)
    ax2.set_title(f'{name} - Résidus vs Prédictions', fontsize=12, fontweight='bold')
    ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nAnalyse des résidus :")
print("\n📈 Histogramme :")
print("  - Centré sur 0 : le modèle n'est pas biaisé")
print("  - Forme en cloche (normale) : hypothèse de normalité vérifiée")
print("\n📈 Résidus vs Prédictions :")
print("  - Points aléatoires autour de 0 : bon modèle (pas de pattern)")
print("  - Pattern visible (courbe, tendance) : modèle ne capture pas toute la complexité")
print("  - Variance croissante : hétéroscédasticité (problème potentiel)")

## 8. Impact des Outliers sur les Métriques

Créons un dataset avec outliers pour comprendre la différence entre RMSE et MAE.

In [ ]:
# Créer des données synthétiques
np.random.seed(42)
n_samples = 100

# Vraies valeurs
y_true = np.linspace(0, 100, n_samples)

# Prédictions avec erreur normale
y_pred_normal = y_true + np.random.normal(0, 5, n_samples)

# Prédictions avec quelques outliers (erreurs très importantes)
y_pred_outliers = y_pred_normal.copy()
outlier_indices = [10, 25, 50, 75, 90]
y_pred_outliers[outlier_indices] += np.random.choice([-40, -30, 30, 40], size=len(outlier_indices))

print("Dataset synthétique créé :")
print(f"  - {n_samples} exemples")
print(f"  - Sans outliers : erreur normale (std=5)")
print(f"  - Avec outliers : {len(outlier_indices)} erreurs extrêmes ajoutées")

In [ ]:
# Calcul des métriques avec et sans outliers
metrics_comparison = []

for label, y_pred in [('Sans outliers', y_pred_normal), ('Avec outliers', y_pred_outliers)]:
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    metrics_comparison.append({
        'Cas': label,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'Ratio RMSE/MAE': rmse / mae
    })

df_comparison = pd.DataFrame(metrics_comparison)

print("\n" + "=" * 80)
print("IMPACT DES OUTLIERS SUR LES MÉTRIQUES")
print("=" * 80)
print(df_comparison.to_string(index=False))

# Calcul des variations
mse_increase = (df_comparison.loc[1, 'MSE'] - df_comparison.loc[0, 'MSE']) / df_comparison.loc[0, 'MSE'] * 100
rmse_increase = (df_comparison.loc[1, 'RMSE'] - df_comparison.loc[0, 'RMSE']) / df_comparison.loc[0, 'RMSE'] * 100
mae_increase = (df_comparison.loc[1, 'MAE'] - df_comparison.loc[0, 'MAE']) / df_comparison.loc[0, 'MAE'] * 100

print("\n📊 ANALYSE :")
print(f"  - MSE augmente de {mse_increase:.1f}% avec les outliers (très sensible !)")
print(f"  - RMSE augmente de {rmse_increase:.1f}% avec les outliers (sensible)")
print(f"  - MAE augmente de {mae_increase:.1f}% avec les outliers (robuste)")
print("\n💡 CONCLUSION :")
print("  - MAE est beaucoup plus robuste aux outliers que RMSE/MSE")
print("  - Le ratio RMSE/MAE indique la présence d'outliers :")
print(f"    • Sans outliers : {df_comparison.loc[0, 'Ratio RMSE/MAE']:.2f} (proche de 1)")
print(f"    • Avec outliers : {df_comparison.loc[1, 'Ratio RMSE/MAE']:.2f} (> 1.5 = outliers détectés)")

In [ ]:
# Visualisation de l'impact des outliers
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Sans outliers
ax = axes[0]
ax.scatter(y_true, y_pred_normal, alpha=0.6, s=50, label='Prédictions', color='blue')
ax.plot([0, 100], [0, 100], 'r--', linewidth=2, label='Prédiction parfaite')
ax.set_xlabel('Vraies Valeurs', fontsize=12)
ax.set_ylabel('Prédictions', fontsize=12)
ax.set_title(f'Sans Outliers\nRMSE={df_comparison.loc[0, "RMSE"]:.2f}, MAE={df_comparison.loc[0, "MAE"]:.2f}', 
             fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Avec outliers
ax = axes[1]
ax.scatter(y_true, y_pred_outliers, alpha=0.6, s=50, label='Prédictions', color='green')
ax.scatter(y_true[outlier_indices], y_pred_outliers[outlier_indices], 
           color='red', s=150, marker='X', edgecolors='black', linewidth=2, 
           label='Outliers', zorder=5)
ax.plot([0, 100], [0, 100], 'r--', linewidth=2, label='Prédiction parfaite')
ax.set_xlabel('Vraies Valeurs', fontsize=12)
ax.set_ylabel('Prédictions', fontsize=12)
ax.set_title(f'Avec Outliers\nRMSE={df_comparison.loc[1, "RMSE"]:.2f}, MAE={df_comparison.loc[1, "MAE"]:.2f}', 
             fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nVisuellement, on voit que :")
print("  - Les outliers (croix rouges) s'écartent beaucoup de la ligne de prédiction parfaite")
print("  - RMSE augmente plus que MAE car il met au carré les erreurs (pénalise les grandes erreurs)")
print("  - MAE traite toutes les erreurs de manière proportionnelle (plus robuste)")

## 9. Guide de Sélection des Métriques

In [ ]:
print("\n" + "=" * 80)
print("GUIDE DE SÉLECTION DES MÉTRIQUES DE RÉGRESSION")
print("=" * 80)

print("\n🎯 MSE (Mean Squared Error)")
print("   ✅ Quand utiliser :")
print("      - Optimisation mathématique (dérivable, convexe)")
print("      - Grandes erreurs très pénalisantes (finance, sécurité)")
print("   ❌ Éviter si :")
print("      - Dataset contient des outliers")
print("      - Besoin d'interprétabilité (unité au carré)")

print("\n🎯 RMSE (Root Mean Squared Error)")
print("   ✅ Quand utiliser :")
print("      - Même unité que la cible (facile à interpréter)")
print("      - Grandes erreurs plus pénalisantes")
print("      - Standard pour la régression (défaut)")
print("   ❌ Éviter si :")
print("      - Dataset contient beaucoup d'outliers")

print("\n🎯 MAE (Mean Absolute Error)")
print("   ✅ Quand utiliser :")
print("      - Dataset avec outliers")
print("      - Toutes les erreurs ont un coût similaire")
print("      - Besoin de robustesse")
print("   ❌ Éviter si :")
print("      - Grandes erreurs doivent être fortement pénalisées")

print("\n🎯 R² (Coefficient de Détermination)")
print("   ✅ Quand utiliser :")
print("      - Comparer des modèles (indépendant de l'échelle)")
print("      - Mesurer la proportion de variance expliquée")
print("      - Communication (facile à expliquer : 0-100%)")
print("   ❌ Éviter si :")
print("      - Seul critère (peut masquer des problèmes)")
print("      - Beaucoup de features (préférer R² ajusté)")

print("\n🎯 MAPE (Mean Absolute Percentage Error)")
print("   ✅ Quand utiliser :")
print("      - Comparer différents datasets (indépendant de l'échelle)")
print("      - Interprétation en pourcentage (business)")
print("   ❌ Éviter si :")
print("      - Cible contient des 0 (division par 0)")
print("      - Asymétrie problématique (pénalise plus surestimation)")

print("\n" + "=" * 80)
print("💡 RECOMMANDATION GÉNÉRALE :")
print("   Toujours reporter PLUSIEURS métriques (ex: RMSE + MAE + R²)")
print("   Cela donne une vision complète de la performance du modèle.")
print("=" * 80)

## 10. Résumé et Conclusions

In [ ]:
print("\n" + "=" * 80)
print("RÉSUMÉ DU NOTEBOOK - MÉTRIQUES DE RÉGRESSION")
print("=" * 80)

print("\n✅ Ce que nous avons appris :")
print("\n1. MSE vs RMSE vs MAE :")
print("   - MSE pénalise fortement les grandes erreurs (au carré)")
print("   - RMSE est plus interprétable (même unité que y)")
print("   - MAE est robuste aux outliers")

print("\n2. R² mesure la variance expliquée :")
print("   - R² = 1 : modèle parfait")
print("   - R² = 0 : modèle équivalent à prédire la moyenne")
print("   - R² < 0 : modèle très mauvais")

print("\n3. MAPE donne une erreur en pourcentage :")
print("   - Indépendant de l'échelle")
print("   - Attention aux divisions par 0")

print("\n4. Les résidus donnent des informations précieuses :")
print("   - Distribution normale centrée sur 0 : bon modèle")
print("   - Pattern dans les résidus : modèle incomplet")

print("\n5. Le ratio RMSE/MAE détecte les outliers :")
print("   - Proche de 1 : peu d'outliers")
print("   - > 1.5 : présence d'outliers significatifs")

print("\n6. Toujours reporter PLUSIEURS métriques :")
print("   - RMSE + MAE + R² est un bon trio par défaut")
print("   - Chaque métrique apporte une information différente")

print("\n" + "=" * 80)
print("📚 Prochaine étape : Validation croisée pour une évaluation robuste")
print("=" * 80)

## Exercices

1. Calculez le R² ajusté pour chaque modèle. Lequel reste le meilleur ?

2. Créez un dataset où RMSE >> MAE et expliquez pourquoi.

3. Implémentez vous-même les fonctions `mse()`, `mae()`, `r2_score()` en NumPy pur.

4. Tracez la courbe de learning (train vs test scores) pour détecter l'overfitting.

5. Comparez MAPE avec sMAPE (symmetric MAPE) qui corrige l'asymétrie.